# Exploratory NLP on chat messages in pilot data

To do:
-- try different tokenizer to deal with punctuation?
-- bag of tower model (tower pairs: group L/C and C/L)
    for each pair, compare first and last occurance of a specific BOT
    interested in words that drop out and words that increase in frequency
-- fix dictionary to be union not intersection
-- TFIDF for tower specific 
    what are the specific words we use to describe tower pairs
-- condition on rep 1, use same langauge for all three items, do subjects disambiguate by the later trials

-- compare probability of words for L-pi and L-C in early and late stages (factor out the "L") 

[X] try tf-idf with LC as doc 1 and LP,CP combined as doc 2 in final rep 

https://en.wikipedia.org/wiki/Pointwise_mutual_information
    
calculate PMI for all words for all dyads

for each dyad, get PMI distribution for each towerset permutation (compare PL/CP, PL/CL, CP/CL) in first rep 
take average of this PMI (so three summary measures) then do this again for the last rep

TF-IDF first and last

-------------------------
Spacy lemmatizer, numbers to text

Calculate new column (prop of L specifc over all words in rep)

Proportion of "L" specific words in LP and LC (i.e. changing the denominator to only be towersets with L) -- then average for each dyad across the two towers

Also column for raw count of L specifc words

plot histogram of differences between rep 0 and rep 3



In [ ]:
#!pip install spacy

## Import packages

In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from collections import Counter
from num2words import num2words
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import matutils, models
import matplotlib.pyplot as plt
import spacy

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
pd.set_option('display.max_colwidth', 1000)
import pickle
import re
import math


stemmer = SnowballStemmer("english")

## Import data frames (generated in DF_generator script from mongoDB)

In [2]:
df_chat = pd.read_csv('df_chat.csv')
print(df_chat.shape)
print("Dyads:",len(df_chat.gameid.unique()))

(3066, 21)
Dyads: 71


### Check exclusion criteria

In [3]:
df_trial = pd.read_csv('df_trial.csv')
df75 = pd.DataFrame(df_trial.groupby(['gameid', 'trialNum'])['trialScore'].sum()>75).groupby(['gameid']).sum()
df75['trials'] = df75['trialScore']

#delete rows that aren't greater than 75% in 9 trials
df75 = df75[df75['trials']>=9]

print("Total dyads achieving 75% Accuracy on 75% of trials:",len(df75))

Total dyads achieving 75% Accuracy on 75% of trials: 48


### Remove dyads that did not pass exclusion criteria

In [4]:
keep_id = df75.index.tolist()
keep_id
df = df_chat[df_chat['gameid'].isin(keep_id)]
print(df.shape)
print("Passed Dyads:",len(df.gameid.unique()))

(2223, 21)
Passed Dyads: 48


In [5]:
df = df[['gameid','repNum','content','trialNum', 'leftTarget', 'rightTarget']]
df = df[df.trialNum != 'practice']
df = df[df.repNum != 'practice']
df['repNum'] = df['repNum'].astype(int)
df['C'] = (df['leftTarget'] + df['rightTarget']).apply(lambda x: 1 if 'C' in x   else 0)
df['L'] = (df['leftTarget'] + df['rightTarget']).apply(lambda x: 1 if 'L' in x   else 0)
df['P'] = (df['leftTarget'] + df['rightTarget']).apply(lambda x: 1 if 'Pi' in x   else 0)
df = df.drop(['leftTarget', 'rightTarget'],axis = 1)
df.head(2)


,gameid,repNum,content,trialNum,C,L,P
2,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,0,put a blue block three from the left~~~,0,1,0,1
3,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,0,place a red block on top of the left side of the blue block and then another red on top of that red,0,1,0,1


## Create Bag of Words for each repNum by concatenating content
create df with column for repnum and column for entire string grouped by repnum

In [6]:
# df['BOW'] = df[['repNum','C','L','Pi','content']].groupby(['repNum','C','L','Pi'])['content'].transform(lambda x: ' '.join(x))
# df = df[['repNum','C','L','Pi','BOW']]
df = df.groupby(['gameid','repNum','C','L','P'])['content'].apply(lambda x: ' '.join(x)).reset_index()

print(df.shape)
df.head(2)

(576, 6)


,gameid,repNum,C,L,P,content
0,0110-5784fec9-109a-4d7a-a343-4820f4d42144,0,0,1,1,"two blue blocks on left and right side of left right line two red block on top of left blue block just placed down two red block one space apart on right line one blue block on top red block on left red line, on more red block on top of left red block"
1,0110-5784fec9-109a-4d7a-a343-4820f4d42144,0,1,0,1,"for left red line, place blue block in the middle then stack two red on left side of that blue block for left red line, place blue block on highest red so it stick out on the right side for right red line, place one red block two space away from that line, same as step 3, but on the left side, so there is two spaces apart between those two red blocks for right red line, place one blue block on top of each red block so the blue blocks meet/touch"


### Remove stopwords and punctuation

In [7]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['BOW_clean'] = df['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [11]:
df['BOW_clean'] = df['BOW_clean'].str.lower()
df['BOW_clean'] = df['BOW_clean'].str.replace(r'~', '')
df['BOW_clean'] = df['BOW_clean'].str.replace(r'\(', '')
df['BOW_clean'] = df['BOW_clean'].str.replace(r'\)', '')
df['BOW_clean'] = df['BOW_clean'].str.replace(r'\,', '')
df['BOW_clean'] = df['BOW_clean'].str.replace(r"\'", '')
df['BOW_clean'] = df['BOW_clean'].str.replace(r"\:", '')
df['BOW_clean'] = df['BOW_clean'].str.replace(r'"', '')
df['BOW_clean'] = df['BOW_clean'].str.replace(r"-", '')
df['BOW_clean'] = df['BOW_clean'].str.replace(r"/", ' ')

## Convert numbers to words

In [12]:
import num2words
from num2words import num2words
def num_2_words(sentence):
    out = ""
    for word in sentence.split():
        try:
            o = num2words(word)
        except:
            o = word
        out = out+" "+ o
    return out

    

In [13]:
df['BOW_clean'] = df['BOW_clean'].apply(lambda x: num_2_words(x))
df.head(2)

,gameid,repNum,C,L,P,content,BOW_clean
0,0110-5784fec9-109a-4d7a-a343-4820f4d42144,0,0,1,1,"two blue blocks on left and right side of left right line two red block on top of left blue block just placed down two red block one space apart on right line one blue block on top red block on left red line, on more red block on top of left red block",two blue blocks left right side left right line two red block top left blue block placed two red block one space apart right line one blue block top red block left red line red block top left red block
1,0110-5784fec9-109a-4d7a-a343-4820f4d42144,0,1,0,1,"for left red line, place blue block in the middle then stack two red on left side of that blue block for left red line, place blue block on highest red so it stick out on the right side for right red line, place one red block two space away from that line, same as step 3, but on the left side, so there is two spaces apart between those two red blocks for right red line, place one blue block on top of each red block so the blue blocks meet/touch",left red line place blue block middle stack two red left side blue block left red line place blue block highest red stick right side right red line place one red block two space away line step three left side two spaces apart two red blocks right red line place one blue block top red block blue blocks meet touch


In [14]:
# # Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
# nlp = spacy.load('en', disable=['parser', 'ner'])

# sentence = "The striped bats are hanging on their feet for best"

# # Parse the sentence using the loaded 'en' model object `nlp`
# doc = nlp(sentence)

# # Extract the lemma for each token and join
# " ".join([token.lemma_ for token in doc]).split()

In [15]:
# df['BOW_lemmatized'] = df['BOW_clean'].apply(lambda x: " ".join([token.lemma_ for token in nlp(x)]).split())


In [16]:
import nltk
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in tokenizer.tokenize(text)]
df['BOW_lemmatized'] = df['BOW_clean'].apply(lemmatize_text)
print(df.shape)
df.head(3)

(576, 8)


,gameid,repNum,C,L,P,content,BOW_clean,BOW_lemmatized
0,0110-5784fec9-109a-4d7a-a343-4820f4d42144,0,0,1,1,"two blue blocks on left and right side of left right line two red block on top of left blue block just placed down two red block one space apart on right line one blue block on top red block on left red line, on more red block on top of left red block",two blue blocks left right side left right line two red block top left blue block placed two red block one space apart right line one blue block top red block left red line red block top left red block,"[two, blue, block, left, right, side, left, right, line, two, red, block, top, left, blue, block, placed, two, red, block, one, space, apart, right, line, one, blue, block, top, red, block, left, red, line, red, block, top, left, red, block]"
1,0110-5784fec9-109a-4d7a-a343-4820f4d42144,0,1,0,1,"for left red line, place blue block in the middle then stack two red on left side of that blue block for left red line, place blue block on highest red so it stick out on the right side for right red line, place one red block two space away from that line, same as step 3, but on the left side, so there is two spaces apart between those two red blocks for right red line, place one blue block on top of each red block so the blue blocks meet/touch",left red line place blue block middle stack two red left side blue block left red line place blue block highest red stick right side right red line place one red block two space away line step three left side two spaces apart two red blocks right red line place one blue block top red block blue blocks meet touch,"[left, red, line, place, blue, block, middle, stack, two, red, left, side, blue, block, left, red, line, place, blue, block, highest, red, stick, right, side, right, red, line, place, one, red, block, two, space, away, line, step, three, left, side, two, space, apart, two, red, block, right, red, line, place, one, blue, block, top, red, block, blue, block, meet, touch]"
2,0110-5784fec9-109a-4d7a-a343-4820f4d42144,0,1,1,0,"for left red line, two blue block placed right against each other for left red line, on top of far left of left blue block, place two red blocks stacked on each other for right red line, place one blue block in middle of that line for right red line, place two red blocks stacked on top of each other on left side of blue block for right red line, place blue block on top of highest red block so it sticks on on the left side",left red line two blue block placed right left red line top far left left blue block place two red blocks stacked right red line place one blue block middle line right red line place two red blocks stacked top left side blue block right red line place blue block top highest red block sticks left side,"[left, red, line, two, blue, block, placed, right, left, red, line, top, far, left, left, blue, block, place, two, red, block, stacked, right, red, line, place, one, blue, block, middle, line, right, red, line, place, two, red, block, stacked, top, left, side, blue, block, right, red, line, place, blue, block, top, highest, red, block, stick, left, side]"


## Calculate PMI for each rep 0/ rep 3 tower pairing within each dyad
For each dyad, get PMI distribution for each towerset permutation (compare PL/CP, PL/CL, CP/CL) in first rep take average of this PMI (so three summary measures) then do this again for the last rep

log [ p(word | target ) / p(word) ]

In [ ]:
df.gameid.unique()[:10]

In [ ]:
df_grouped = df.groupby(['gameid','repNum'])['BOW_clean'].apply(lambda x: ' '.join(x)).reset_index()
df_grouped.head(1)

In [ ]:
# tower1 = 'C'
# tower2 = 'P'
# tower3 = 'L'
# game = '0208-b497c845-b076-45c9-b958-a62a3ae6e65f'
# set1 = df.loc[(df[tower1] == 1) & (df[tower2] == 1) & (df['gameid'] == game), 'BOW_clean'].iloc[0].split()
# set2 = df_grouped.loc[(df['gameid'] == game) & (df['repNum'] == 3), 'BOW_clean'].iloc[0].split()

# counts1 = Counter(set1)
# total = sum(counts1.values())
# d1 = {k: v / total for k, v in counts.items()}
# #Get word counts for second set
# counts2 = Counter(set2)
# total = sum(counts2.values())
# d2 = {k: v / total for k, v in counts.items()}

# counts_combined = Counter(set_combined)
# total = sum(counts_combined.values())
# d_combined = {k: v / total for k, v in counts.items()}


In [ ]:
df.columns

In [ ]:
reps = [0,3]

gameids = df['gameid'].unique()

set1 = df.loc[(df[tower1] == 1) & (df[tower2] == 1) & (df['gameid'] == game), 'BOW_clean'].iloc[rep].split()
set_combined =  df_grouped.loc[(df_grouped['gameid'] == game) & (df_grouped['repNum'] == rep), 'BOW_clean'].iloc[0].split()

counts = Counter(set1)
total = sum(counts.values())
d1 = {k: v / total for k, v in counts.items()}
counts = Counter(set_combined)
total = sum(counts.values())
d2 = {k: v / total for k, v in counts.items()}
# countsn = Counter(n)
# total = sum(countsn.values())
# dn = {k: v / total for k, v in countsn.items()}

#d_diff = {key: countsl[key] / countsn.get(key, 0) for key in countsl}
out = {key: d1[key] / d2.get(key, 0)  for key in d1}
out = {key: np.log(float(out[key])) for key in out}
print(sum(out.values()) / float(len(out)))
out

In [ ]:
tower1 = 'P'
tower2 = 'L'

reps = [0,3]
gameids = df['gameid'].unique()

rep_out = []
game_out = []
MI_out = []
dict_out = []
results = []
for game in gameids:
    for rep in reps:
        set1 = df.loc[(df[tower1] == 1) & (df[tower2] == 1) & (df['gameid'] == game), 'BOW_clean'].iloc[rep].split()
        set_combined =  df_grouped.loc[(df_grouped['gameid'] == game) & (df_grouped['repNum'] == rep), 'BOW_clean'].iloc[0].split()

        counts = Counter(set1)
        total = sum(counts.values())
        d1 = {k: v / total for k, v in counts.items()}
        counts = Counter(set_combined)
        total = sum(counts.values())
        d2 = {k: v / total for k, v in counts.items()}
        # countsn = Counter(n)
        # total = sum(countsn.values())
        # dn = {k: v / total for k, v in countsn.items()}

        #d_diff = {key: countsl[key] / countsn.get(key, 0) for key in countsl}
        out = {key: d1[key] / d2.get(key, 0)  for key in d1}
        out = {key: np.log(float(out[key])) for key in out}
        mutual_information = sum(out.values()) / float(len(out))

#         game_out.append(game)
#         rep_out.append(rep)
#         MI_out.append(mutual_information)
#         dict_out.append(out)
        results.append([game, rep, mutual_information, out])
  
  
# Create the pandas DataFrame  
df_mi = pd.DataFrame(results, columns = ['gameid', 'repNum','MI','PMI'])

print(df_mi.shape)
df_mi.head(2)

In [ ]:
PMI_0 = []
for d in df_mi[df_mi['repNum'] == 0]['PMI']:
    PMI_0.append(list(d.values()))
PMI_0 = [ item for elem in PMI_0 for item in elem]


PMI_3 = []
for d in df_mi[df_mi['repNum'] == 3]['PMI']:
    PMI_3.append(list(d.values()))
PMI_3 = [ item for elem in PMI_3 for item in elem]

In [ ]:
plt.figure(figsize=(8,6))
plt.hist(PMI_0, bins=50, alpha=0.5, label="PMI_0")
plt.hist(PMI_3, bins=50, alpha=0.5, label="PMI_3")

plt.xlabel("PMI", size=14)
plt.ylabel("Count", size=14)
plt.title("PMI for P/L in rep 0 and rep 3")
plt.legend(loc='upper right')

In [ ]:
d = {'repNum':'C/P Towerset Mutual information'}
df1 = df_mi.groupby('repNum')['MI'].agg(['mean','sem']).reset_index()

fig, ax = plt.subplots()
ax.bar(data = df1, x = 'repNum', height = 'mean', yerr='sem')
plt.title("C/L Towerset Mutual information")
plt.ylabel("Mutual Information")
plt.xlabel("RepNum")
#plt.errorbar(data = df1, x='repNum', y='MI') 

# plt.yscale("log") 
# plt.xlabel("Genre") 

## Isolate words for specific tower within rep and within dyad

In [ ]:
#ISOLATE TOWER 1
tower1 = 'C'
tower2 = 'P'
tower3 = 'L'

reps = [0,3]
gameids = df['gameid'].unique()

rep_out = []
game_out = []
MI_out = []
dict_out = []
results = []
for game in gameids:
    for rep in reps:
        target_words_1 = df.loc[(df[tower1] == 1) & (df[tower2] == 1) & (df['gameid'] == game), 'BOW_clean'].iloc[rep].split()
        target_words_2 = df.loc[(df[tower1] == 1) & (df[tower3] == 1) & (df['gameid'] == game), 'BOW_clean'].iloc[rep].split()
        non_target_words = df.loc[(df[tower2] == 1) & (df[tower3] == 1) & (df['gameid'] == game), 'BOW_clean'].iloc[rep].split()
        
        target_specific_1 = list(set(target_words_1) - set(non_target_words))
        target_specific_2 = list(set(target_words_2) - set(non_target_words))
        specific_words = list(set(target_specific_1) & set(target_specific_2))


        results.append([game, rep, tower1, specific_words])
  
  
# # Create the pandas DataFrame  
df_intersection = pd.DataFrame(results, columns = ['gameid', 'repNum','tower','words'])

print(df_intersection.shape)
df_intersection.head(10)


In [ ]:
s = df_intersection[df_intersection['repNum'] == 3]['words']
l = [element for list_ in s for element in list_]
counts = Counter(l)
print(len(counts.keys()))
counts.most_common()


In [ ]:
labels, values = zip(*counts.items())

# sort your values in descending order
indSort = np.argsort(values)[::-1]

from matplotlib.pyplot import figure
figure(num=None, figsize=(10, 5), dpi=80, facecolor='w', edgecolor='k')
x_limit = 30

# rearrange your data
labels = np.array(labels)[indSort][:x_limit]
values = np.array(values)[indSort][:x_limit]

indexes = np.arange(len(labels))

bar_width = 0.35

plt.bar(indexes, values)

# add labels
plt.xticks(indexes + bar_width, labels,  rotation='vertical')
plt.show()

In [ ]:
df1

### Build a matrix of each word in the corpus and its TF-IDF for that comment

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = df['BOW_clean']
vectorizer = TfidfVectorizer(sublinear_tf = True)
vectors = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df_vectors = pd.DataFrame(denselist, columns=feature_names)

### Select the top 5 words (TF-IDF) for each comment
##### Intuitively, this roughly corresponds to what this person is talking about relatively more than any other person

In [ ]:
Tops = pd.DataFrame(df_vectors.apply(lambda x:list(df_vectors.columns[np.array(x).argsort()[::-1][:15]]), axis=1).to_list())

Tops

## Okay, that was a dud, let's just plot histograms of word frequency for each repNum

## Build Towerset and dyad specific BOW separated by first and last rep

In [ ]:
tower1 = 'C'
tower2 = 'L'
word_list = []
gameid = df['gameid'].unique()
for game in gameid:
    set1 = df.loc[(df[tower1] == 1) & (df[tower2] == 1) & (df['gameid'] == game), 'BOW_clean'].iloc[0].split()
    set2 = df.loc[(df[tower1] == 1) & (df[tower2] == 1) & (df['gameid'] == game), 'BOW_clean'].iloc[3].split()
    #Get word counts for first set
    counts = Counter(set1)
    total = sum(counts.values())
    d1 = {k: v / total for k, v in counts.items()}
    #Get word counts for second set
    counts = Counter(set2)
    total = sum(counts.values())
    d2 = {k: v / total for k, v in counts.items()}
    labels, values = zip(*counts.items())

    #Get difference in count dicts
    d_diff = {key: d2[key] - d1.get(key, 0) for key in d2}
    words = list({k: v for k, v in sorted(d_diff.items(), key=lambda item: item[1], reverse = True)}.items())[:5]
    word_list.append(words)


In [ ]:
out = pd.DataFrame({'gameid':gameid,'word_list':word_list})

In [ ]:
print(out.shape)
out

## WORD CLOUD!
perhaps pointless, but definitely fun!

In [ ]:
from wordcloud import WordCloud
figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
wordcloud = WordCloud().generate(df['BOW_clean'].iloc[1])

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()


## Try TF-IDF with tower specific docs

In [ ]:
CP_3 = df.loc[(df['C'] == 1) & (df['P'] == 1), 'BOW_clean'].iloc[3]
CL_3 = df.loc[(df['C'] == 1) & (df['L'] == 1), 'BOW_clean'].iloc[3]
LP_3 = df.loc[(df['L'] == 1) & (df['P'] == 1), 'BOW_clean'].iloc[3]

doc1 = "hello this this this this this two five ten"
doc2 = "Other thing this this this this this two hello else"

vectorizer = TfidfVectorizer(sublinear_tf = True)
vectors = vectorizer.fit_transform([doc1,doc2])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df_vectors = pd.DataFrame(denselist, columns=feature_names)

Tops = pd.DataFrame(df_vectors.apply(lambda x:list(df_vectors.columns[np.array(x).argsort()[::-1][:15]]), axis=1).to_list())

Tops

In [ ]:
len(doc1)